In [1]:
import dataset
import dtw_recognition
import numpy as np

# DTW

In [2]:
config = {
    "data_dir": "../data/05/19021396_PhamThanhVinh",
    "sr": 22050,
    "n_mfcc": 13,
    "hop_length": 256,
    "n_fft": 512,
    "delta_width": 5,
    "n_sample_per_word": 3,
}
df = dataset.get_dataset_df(config["data_dir"], config["sr"], config["n_mfcc"], config["hop_length"], config["n_fft"],
                   config["delta_width"])
template_df = dataset.get_template_df(df, config["n_sample_per_word"])

In [3]:
df

,label,mfcc_features
0,sil,"[[0.0, 0.36298612, 1.0, 0.67046523, 1.0, 0.109..."
1,1,"[[0.0, 0.075219214, 0.5102628, 0.37972867, 0.6..."
2,tram,"[[0.29391974, 0.69984466, 0.7457707, 0.3506259..."
3,4,"[[0.9442609, 0.0, 0.50626904, 0.52150154, 0.72..."
4,muoi,"[[1.0, 0.54144883, 0.37757295, 0.39642286, 0.0..."
...,...,...
1975,tram,"[[0.26855448, 0.39782253, 0.06277703, 0.511701..."
1976,8,"[[0.006426136, 0.03151538, 1.0, 0.63716173, 0...."
1977,muoi,"[[1.0, 0.16773497, 1.0, 0.20140655, 0.2975387,..."
1978,tu,"[[0.3128125, 0.8595457, 0.54761374, 0.0, 0.365..."


In [4]:
# avg = {}
# min = {}
# max = {}
#
# syllables = pd.DataFrame(columns=["label", "n_syllables"])
# syllables["label"] = df["label"].unique()
# syllable_list = []
#
# for label in df["label"].unique():
#     dfi = df[df["label"] == label]
#     avg_i = 0
#     min_i = np.inf
#     max_i = 0
#     for i in range(len(dfi)):
#         a = dfi["mfcc_features"].iloc[i].shape[0]
#         avg_i += a
#         if a < min_i:
#             min_i = a
#         if a > max_i:
#             max_i = a
#     avg[label] = int(avg_i/len(dfi))
#     min[label] = min_i
#     max[label] = max_i
#
# rate = 0.6
# for i in range(len(syllables)):
#     s_i = int(min[syllables["label"].iloc[i]]*rate)
#     if s_i >= 6:
#         s_i -= 3
#     syllable_list.append(s_i)
# syllables["n_syllables"] = syllable_list
#
# syllables.to_csv("../data/syllables.csv", sep="\t", header=None, index=False)

In [5]:
template_df

,label,mfcc_features
0,sil,"[[1.0, 0.25591674, 1.0, 0.0, 0.5374895, 0.0, 0..."
1,sil,"[[1.0, 0.005836276, 1.0, 0.0, 0.18818209, 0.26..."
2,sil,"[[1.0, 0.0, 1.0, 0.0, 0.14333844, 0.0, 1.0, 0...."
3,1,"[[0.008115292, 0.13720374, 0.6464415, 0.0, 0.0..."
4,1,"[[0.08613336, 0.04695834, 0.72694945, 0.607525..."
5,1,"[[0.0, 0.0, 0.0, 0.5741292, 1.0, 0.92259985, 0..."
6,tram,"[[0.092639014, 0.56376374, 1.0, 0.6377285, 0.7..."
7,tram,"[[0.5384278, 0.25761774, 0.8564208, 0.7418925,..."
8,tram,"[[0.0, 0.31076217, 0.4893239, 0.58495325, 0.48..."
9,4,"[[0.0, 0.0, 0.29382217, 0.48106688, 0.50523496..."


In [6]:
# new_template_df = dtw_recognition.reduce_models(template_df, "../data/syllables.csv")

In [7]:
choice_indexes = np.random.choice(len(df), 100, replace=False)
test_mfcc_features = list(df.loc[choice_indexes, "mfcc_features"])

In [8]:
df.iloc[choice_indexes]

,label,mfcc_features
1803,muoi,"[[0.45470503, 0.0, 1.0, 0.25042507, 0.04882723..."
1855,7,"[[0.02119631, 0.07551253, 0.62779695, 0.493811..."
206,3,"[[0.5425608, 0.056398604, 1.0, 1.0, 0.83498853..."
1723,mot,"[[0.9057198, 0.0, 0.6648663, 0.8217759, 0.9277..."
1677,sil,"[[1.0, 0.0, 1.0, 0.6383448, 1.0, 0.69336957, 1..."
...,...,...
1742,muoi,"[[1.0, 0.0, 1.0, 0.2877752, 0.19795953, 1.0, 1..."
464,5,"[[0.20449713, 0.9357886, 0.82763886, 0.9766568..."
23,tram,"[[0.5111927, 0.4801271, 0.6735646, 0.25744382,..."
1930,8,"[[0.0, 0.11004019, 1.0, 1.0, 0.92647606, 0.380..."


In [9]:
preds = dtw_recognition.predict(template_df, test_mfcc_features)

In [10]:
print(f'Accuracy: {np.sum(preds == df.loc[choice_indexes, "label"])/len(preds)}')

Accuracy: 0.83


# HMM